In [1]:
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets,layers,models
import numpy as np
import matplotlib.pyplot as plt


In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()


In [3]:
print('x_train: ', x_train.shape)
print('y_train: ', y_train.shape)

x_train:  (50000, 32, 32, 3)
y_train:  (50000, 1)


In [4]:
K = len(np.unique(y_train)) # Classes
Ntr = x_train.shape[0]
Nte = x_test.shape[0]
Din = 3072 # CIFAR10
print('Noof classes:',K)
print('Noof training images:',Ntr)
print('Noof test images:',Nte)
print('Noof features:',Din)

x_train=x_train[range(Ntr),:]
y_train=y_train[range(Ntr),:]
x_test=x_test[range(Nte),:]
x_test=x_test[range(Nte),:]
# Normalize pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0
mean_image = np.mean(x_train, axis=0)
x_train = x_train - mean_image
x_test = x_test - mean_image

Noof classes: 10
Noof training images: 50000
Noof test images: 10000
Noof features: 3072


In [5]:
print('original labels:',y_train.shape)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=K)
print('labels in class matrix:',y_train.shape)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=K)

original labels: (50000, 1)
labels in class matrix: (50000, 10)


In [6]:
x_train = np.reshape(x_train,(Ntr,Din))
x_test = np.reshape(x_test,(Nte,Din))
print('x_train:', x_train.shape)
print('x_test:', x_test.shape)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train: (50000, 3072)
x_test: (10000, 3072)


In [7]:
H=200
std=1e-5
w1 = std*np.random.randn(Din, H)
w2 = std*np.random.randn(H, K)
b1 = np.zeros(H)
b2=np.zeros(K)
print("w1:", w1.shape)
print("b1:", b1.shape)
print("w2:", w2.shape)
print("b2:", b2.shape)

w1: (3072, 200)
b1: (200,)
w2: (200, 10)
b2: (10,)


In [ ]:
batch_size = Ntr
iterations = 300
lr = 1.4e-2
lr_decay=0.999
reg =5e-6
train_loss_history = []
train_acc_history = []
test_loss_history = []
test_acc_history = []
seed = 0
#rng = np.random.default_rng(seed=seed)
for t in range(iterations):
    indices = np.random.choice(Ntr,batch_size)
    #rng.shuffle(indices)
    x=x_train[indices]
    y=y_train[indices]

    # Forward pass
    z1=1./(1.0 + np.exp(-(x.dot(w1)+b1)))
    y_pred=z1.dot(w2)+b2
    z=1./(1.0 + np.exp(-(x_test.dot(w1)+b1)))
    y_pred_test=z.dot(w2)+b2

    #loss
    loss=(1./batch_size)*np.square(y_pred-y).sum() + reg * np.sum(w1*w1) + reg * np.sum(w2*w2)
    test_loss=1./Nte*(np.square(y_pred_test-y_test)).sum() + reg * np.sum(w1*w1) + reg * np.sum(w2*w2)
    train_loss_history.append(loss)
    test_loss_history.append(test_loss)

    #accuracy
    train_acc = 1.0 - (1/(batch_size*K))*(np.abs(np.argmax(y,axis=1) - np.argmax(y_pred,axis=1))).sum()
    train_acc_history.append(train_acc)
    test_acc = 1.0 - (1/(Nte*K))*(np.abs(np.argmax(y_test,axis=1) - np.argmax(y_pred_test,axis=1))).sum()
    test_acc_history.append(test_acc)

    if t%10==0:
        print('epoch %d/%d: train loss= %f-- ,test loss= %f--,train accuracy= %f--, test accuracy= %f' % (t,iterations,loss,test_loss,train_acc,test_acc))

        
    # Backward pass

    dy_pred=(1./batch_size)*2.0*(y_pred-y)
    dw2=z1.T.dot(dy_pred) + reg * w2
    db2=dy_pred.sum(axis=0)
    dz1=dy_pred.dot(w2.T)
    dw1=x.T.dot(dz1*z1*(1-z1))+reg*w1
    db1=(dz1*z1*(1-z1)).sum(axis=0)


    
    w1 -= lr*dw1
    w2 -= lr*dw2
    b1 -= lr*db1
    b2 -= lr*db2
    lr *= lr_decay




